In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash_table.Format import Format, Scheme, Sign, Symbol
import plotly.express as px
import pandas as pd
import pandas_datareader.data as web
from datetime import date
import time
import datetime
import plotly.graph_objs as go


import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


# Historique des transactions

In [138]:
historiques = pd.read_excel("Transactions BD.xlsx")
mouvements = historiques[(historiques['Symbol']!='Cash')]#&(historiques['Compte']=='CTO')
cash = historiques[(historiques['Symbol']=='Cash')]#&(historiques['Compte']=='CTO') 
#Pour remplir les weekends
periode = pd.DataFrame({"Date":pd.date_range(start=date(2020,2,27),end=datetime.datetime.now())})

historiques['Symbol'] = historiques['Symbol'].apply(lambda x : x.replace(u'\xa0', '').strip())
historiques.drop(columns=['ISN','Flux','Taux de change','TVA','Frais','Brut','Mois'],inplace=True)
historiques = historiques[historiques['Transaction'].isin(["Achat","Vente"])]

In [139]:
historiques.count()[0]

20

# Récupération des tickers Yahoo finance

In [140]:
tickers_cto = pd.DataFrame({'Symbol':['UNIBAIL-RODAM.UNIT', 'JDE PEET_S', 'GECINA', 'ALTAREA', 'ICADE',
                                   'BOSTON PROPERTIES', 'DIGITAL REALTY', 'COVIVIO',
                                   'PALANTIR TECHNOLOG', 'KLEPIERRE', 'VONOVIA', 'INTEL',
                                   'ALIBABA GROUP ADR', 'YATSEN HOLDING LIM', 'MICROSOFT', 'TENCENT',
                                   'BIOGEN', 'FUBOTV'],
                        'ticker':['URW.AS','JDEP.AS','GFC.PA','ALTA.PA','ICAD.PA','BXP','DLR',
                                 'COV.PA','PLTR','LI.PA','VNA.DE','INTC','BABA','YSG','MSFT','TCEHY','BIIB','FUBO'],
                        'currency':['EUR','EUR','EUR','EUR','EUR','USD','USD',
                                 'EUR','USD','EUR','EUR','USD','USD','USD','USD','USD','USD','USD'],
                           'Compte':'CTO'})

In [141]:
tickers_pea = pd.DataFrame({'Symbol':['SAFRAN PROV.ECH.',
                                   'SOCIETE GENERALE', 'RENAULT', 'AIRBUS', 'BOUYGUES', 'DANONE',
                                   'PEUGEOT', 'BIC', 'SODEXO', 'TOTAL', 'FDJ', 'THALES', 'ORPEA',
                                   'HEINEKEN', 'AXA', 'RUBIS', 'AROUNDTOWN', 'VINCI', 'LEGRAND',
                                   'KORIAN', 'ROYAL DUTCH SHELL', 'SAP', 'SOPRA STERIA GROUP',
                                   'FOUNTAINE PAJOT', 'EURONEXT','JUST EAT TAKEAWAY','CELLNEX TELECOM',
                                   'STMICROELECTRONICS','UNILEVER','SANOFI','SOITEC','KERING','BIOMERIEUX'],
                            'currency':['EUR','EUR', 'EUR', 'EUR', 'EUR', 'EUR','EUR', 'EUR', 'EUR', 'EUR', 'EUR', 'EUR', 'EUR',
                                        'EUR', 'EUR', 'EUR', 'EUR', 'EUR', 'EUR','EUR', 'EUR', 'EUR', 'EUR',
                                   'EUR', 'EUR','EUR','EUR','EUR','EUR','EUR','EUR','EUR','EUR'],
                            'ticker':['SAF.PA','GLE.PA','RNO.PA','AIR.PA','EN.PA','BN.PA','UG.PA',
                                      'BB.PA','SW.PA','FP.PA','FDJ.PA','HO.PA','ORP.PA',
                                      'HEIA.AS','CS.PA','RUI.PA','AT1.DE','DG.PA','LR.PA','KORI.PA',
                                      'RDSB.AS','SAP.DE','SOP.PA','ALFPC.PA','ENX.PA','TKWY.AS','CLNX.MC',
                                      'STM.PA','UNA.AS','SAN.PA','SOI.PA','KER.PA','BIM.PA'],
                           'Compte':'PEA'})

In [142]:
tickers = pd.concat([tickers_cto,tickers_pea],sort=False)

In [143]:
historiques1 = historiques.merge(tickers[['Symbol','ticker','currency']],on='Symbol',how='inner')#,indicator=True)
historiques1['Date'] = pd.to_datetime(historiques1['Execution'])


# Récupération de l'historique des cours

In [144]:
cours = web.DataReader(historiques1['ticker'].unique()[0],'yahoo',date(2020,2,27),datetime.datetime.now()).reset_index()
cours['ticker'] = historiques1['ticker'].unique()[0]
cours = pd.merge(periode,cours,on='Date',how='left').fillna(method='ffill')

for ticker in historiques1['ticker'].unique()[1:]:
    if ticker != 'UG.PA':
        df = web.DataReader(ticker,'yahoo',date(2020,2,27),datetime.datetime.now()).reset_index()
        df['ticker'] = ticker
        df = pd.merge(periode,df,on='Date',how='left').fillna(method='ffill')
        cours = pd.concat([cours,df])

In [145]:
peugeot = pd.read_csv("cours.csv",index_col=0)
peugeot = peugeot[peugeot['ticker']=='UG.PA']
peugeot['Date'] = pd.to_datetime(peugeot['Date'])

In [146]:
str(datetime.datetime.today().strftime('%Y-%m-%d'))

'2021-02-16'

In [147]:
cours = pd.concat([cours,peugeot])
cours.to_csv("Files/cours{}.csv".format(str(datetime.datetime.today().strftime('%Y-%m-%d'))))
cours.to_csv("Files/Last/cours.csv")

# Construction des positions en portefeuille sur la période

In [148]:
EURO_USD = web.DataReader('EURUSD=X','yahoo',date(2020,2,27),datetime.datetime.now()).reset_index() 
EURO_USD = pd.merge(periode,EURO_USD,on='Date',how='left').fillna(method='ffill')
portfolio = []
for ticker in historiques1.ticker.unique():
    #print(ticker)
    ptf = cours[cours['ticker']==ticker][['Date','Close','ticker']].merge(historiques1[historiques1['ticker']==ticker],on =['ticker','Date'], how='left')
    ptf = ptf.drop(columns=['Compte','Symbol','currency']).merge(tickers[['ticker','Compte','Symbol','currency']],on='ticker',how='left')

    ptf = ptf.merge(EURO_USD[['Date','Close']].rename(columns={'Close':'EUR_USD'}),on='Date',how='left')
    ptf['taux_conversion'] = ptf[['currency','EUR_USD']].apply(lambda x : x[1] if x[0]=='USD' else 1,axis=1)

    ptf['Sens'] = ptf[['Transaction','Quantité']].apply(lambda x : -x[1] if x[0]=='Vente' else x[1],axis=1)
    ptf['Position'] = ptf['Sens'].cumsum(skipna=True)
    ptf['Position'].fillna(method='ffill', inplace= True)
    
    ptf['Cumul_Frais'] = (ptf['TTF']+ptf['Courtage']).cumsum(skipna=True)
    ptf['Cumul_Frais'].fillna(method='ffill', inplace= True)

    ptf['Coût Total'] = (ptf['Cours'] * ptf['Sens']) +ptf['TTF']+ptf['Courtage']
    ptf.loc[ptf['Transaction']=='Achat','Coût Achat'] = (ptf['Cours'] * ptf['Sens']) +ptf['TTF']+ptf['Courtage']

    #    ptf['Coût Filtre'] = ptf[['Coût Achat','Position','Coût Total']].apply(lambda x: 0 if x[1]==0 else (x[0]),axis=1)

    ptf['Coût Filtre'] = ptf[['Position','Coût Total']].apply(lambda x: 0 if x[0]==0 else (x[1]),axis=1)
    
    ptf['Coût Filtre'].fillna(0,inplace=True)
    l=[]
    cumul = 0
    for index,col in ptf[['Position','Coût Filtre','Sens']].iterrows():
        if col[0]==0:# position nulle
            cumul=0
            l.append(cumul)
        elif col[2]<0:# vente
            #col[2] est négative
            # col[0]-col[2] : pour avoir la position juste avant la vente
            cumul = cumul + (cumul/(col[0]-col[2]))*col[2] # + car Sens est négatif dans ce cas
            l.append(cumul)
        elif col[2]>0:#achat
            cumul = cumul+col[1]
            l.append(cumul)

        else:
            l.append(cumul)
    ptf = ptf.join(pd.DataFrame({'Cumul':l}))  
    
    #ptf['Valeur Achat'] = ptf['Coût Achat'].cumsum(skipna=True)
    #ptf['Valeur Achat'].fillna(method='ffill', inplace= True)
    
    ptf['Valeur Marché'] = (ptf['Close']/ptf['taux_conversion']) * ptf['Position'].fillna(0)
    ptf['latent_PL'] = ptf['Valeur Marché'] - ptf['Cumul']
    ptf['PRU'] = ptf[['Cumul','Position','Transaction']].apply(lambda x : x[0]/x[1] if x[2]=='Achat' else np.nan, axis=1)
    
    #ptf.loc[ptf['Position']>0,'PRU1'] = ptf['PRU'].fillna(method='ffill')#, inplace= True)
    
    ptf['PRU1'] = ptf['PRU'].fillna(method='ffill')
    ptf['realised_PL'] = ptf[['PRU1','Cours','Sens','Transaction']].apply(lambda x : (x[0]-x[1])*x[2] if x[3]=='Vente' else np.nan, axis=1)
    ptf['realised_PL_cumul'] = ptf['realised_PL'].cumsum(skipna=True).fillna(method='ffill')
    ptf['realised_PL_cumul'] = ptf['realised_PL_cumul']  - ptf['Cumul_Frais']
    ptf[['Position','PRU1','realised_PL','realised_PL_cumul']] = ptf[['Position','PRU1','realised_PL','realised_PL_cumul']].fillna(0)
    portfolio.append(ptf)

In [149]:
df_portfolio = pd.concat(portfolio).reset_index()

In [150]:
df_portfolio.to_csv("Files/df_portfolio{}.csv".format(str(datetime.datetime.today().strftime('%Y-%m-%d'))))
df_portfolio.to_csv("Files/Last/df_portfolio.csv")

In [151]:
df_portfolio[(df_portfolio['ticker']=='URW.AS')&(df_portfolio['Date']=='2020-11-16')]
#df_portfolio.drop([266,339, 340])

,index,Date,Close,ticker,Execution,Transaction,Quantité,Cours,Courtage,TTF,Compte,Symbol,currency,EUR_USD,taux_conversion,Sens,Position,Cumul_Frais,Coût Total,Coût Achat,Coût Filtre,Cumul,Valeur Marché,latent_PL,PRU,PRU1,realised_PL,realised_PL_cumul


In [152]:
df_portfolio.count()[0]

1424

In [153]:
mask = df_portfolio[['Date','ticker','Quantité']]#.groupby(['Date','ticker']).count().reset_index()
mask['cumcount'] = df_portfolio[['Date','ticker','Quantité']].groupby(['Date','ticker']).cumcount()
mask_ = df_portfolio[['Date','ticker','Quantité']].groupby(['Date','ticker']).count().reset_index()
mask__ = pd.merge(mask_[mask_['Quantité']>1],mask,on=['Date','ticker'],how='inner')#.groupby(['Date','ticker']).max()
mask___ = mask__.groupby(['Date','ticker']).agg({"cumcount":"max"}).reset_index()

index = []
for c in mask___.iterrows():
    doublons = [i for i in mask[(mask['Date']==c[1][0])&(mask['ticker']==c[1][1])&(mask['cumcount']!=c[1][2])].index]
    df_portfolio.drop(doublons,inplace=True)


<ipython-input-153-2185f0d52dfb>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Calculer le solde du cash durant la période

In [154]:
mouvements_par_jour = mouvements[['Execution','Brut']].groupby('Execution').sum().reset_index()

realised_pl = df_portfolio[['Date','realised_PL']].groupby('Date').sum().reset_index()
latent_pl = df_portfolio[['Date','latent_PL']].groupby(['Date']).sum().reset_index()
valeur_marche = df_portfolio[['Date','Valeur Marché']].groupby(['Date']).sum().reset_index()

solde = pd.merge(periode,mouvements_par_jour,left_on='Date',right_on='Execution',how='left')
print(solde.count()[0])
solde = pd.merge(solde,realised_pl,left_on='Date',right_on='Date',how='left')
print(solde.count()[0])
solde = pd.merge(solde,latent_pl,left_on='Date',right_on='Date',how='left')
print(solde.count()[0])
solde = pd.merge(solde,valeur_marche,left_on='Date',right_on='Date',how='left')
print(solde.count()[0])
solde = pd.merge(solde,cash[['Execution','Cours']].rename(columns={'Cours':'Cash'}),left_on='Date',right_on='Execution',how='left')
print(solde.count()[0])

solde['Brut'].fillna(0,inplace=True)
solde['Cash'].fillna(0,inplace=True)
solde['realised_PL'].fillna(0,inplace=True)
solde['latent_PL'].fillna(method="ffill",inplace=True)
solde['Valeur Marché'].fillna(method="ffill",inplace=True)


solde['Brut_cumul'] = solde['Brut'].cumsum(skipna=True)
solde['Cash_cumul'] = solde['Cash'].cumsum(skipna=True)
solde['realised_PL'] = solde['realised_PL'].cumsum(skipna=True)

solde['solde_cash'] = solde['Cash_cumul'] + solde['realised_PL'] - solde['Brut_cumul']
# portefeuille : positions (valeur marché) + cash + unrealised_pl 
solde['portefeuille'] = solde['Valeur Marché'] + solde['solde_cash']
solde[solde['Date']==pd.to_datetime(datetime.date(2020,11,17))]

356
356
356
356
356


,Date,Execution_x,Brut,realised_PL,latent_PL,Valeur Marché,Execution_y,Cash,Brut_cumul,Cash_cumul,solde_cash,portefeuille
264,2020-11-17,NaT,0.0,187.895,104.004966,692.699966,NaT,0.0,397.18,10010.0,9800.715,10493.414966


In [155]:
solde.to_csv("Files/solde{}.csv".format(str(datetime.datetime.today().strftime('%Y-%m-%d'))))
solde.to_csv("Files/Last/solde.csv")

In [156]:
solde_1 = solde[['Date','latent_PL']].rename(columns={'latent_PL':'Valeur'})
solde_1['Solde'] = 'Unrealised PL'
solde_2 = solde[['Date','realised_PL']].rename(columns={'realised_PL':'Valeur'})
solde_2['Solde'] = 'Realised PL'
solde_3 = solde[['Date','Cash_cumul']].rename(columns={'Cash_cumul':'Valeur'})
solde_3['Solde'] = 'Montant investi'
solde_4 = solde[['Date','solde_cash']].rename(columns={'solde_cash':'Valeur'})
solde_4['Solde'] = 'Solde Cash'
solde_5 = solde[['Date','portefeuille']].rename(columns={'portefeuille':'Valeur'})
solde_5['Solde'] = 'Evaluation PTF'
solde_6 = solde[['Date','Valeur Marché']].rename(columns={'Valeur Marché':'Valeur'})
solde_6['Solde'] = 'Valeur Marché'
solde_global = pd.concat([solde_1,solde_2,solde_3,solde_4,solde_5,solde_6],axis=0)

In [157]:
solde_global.to_csv("Files/solde_global{}.csv".format(str(datetime.datetime.today().strftime('%Y-%m-%d'))))
solde_global.to_csv("Files/Last/solde_global.csv")

# Evaluation totale du portefeuille

In [158]:
valeur_totale = df_portfolio[['Date','Compte','Valeur Marché']].groupby(['Date','Compte']).sum().reset_index().rename(columns={'Valeur Marché':'Evaluation'})
df_portfolio1 = pd.merge(df_portfolio,valeur_totale,on=['Date','Compte'],how='left')
#Date d'ouverture du marché
solde1 = solde[solde['Date'].isin(df_portfolio1['Date'].unique())][['Date','solde_cash']].rename(columns={'solde_cash':'Valeur Marché'})
solde1['Compte'] = 'Cash'
solde1['Symbol'] = 'Cash'
df_portfolio2= df_portfolio1.append(solde1,ignore_index=True).fillna(0)
#df_portfolio2[df_portfolio2['Compte']=='Cash'] = df_portfolio2.loc[df_portfolio2['Compte']=='Cash',:].fillna(0)
#df_portfolio2=df_portfolio1

In [159]:
df_portfolio2[(df_portfolio2['Date']==pd.to_datetime(datetime.date(2020,9,19)))]

,index,Date,Close,ticker,Execution,Transaction,Quantité,Cours,Courtage,TTF,Compte,Symbol,currency,EUR_USD,taux_conversion,Sens,Position,Cumul_Frais,Coût Total,Coût Achat,Coût Filtre,Cumul,Valeur Marché,latent_PL,PRU,PRU1,realised_PL,realised_PL_cumul,Evaluation
205,205.0,2020-09-19,31.110001,FP.PA,0,0,0.0,0.0,0.0,0.0,PEA,TOTAL,EUR,1.18482,1.00000,0.0,20.0,9.72,0.0,0.0,0.0,643.82,622.200012,-21.619988,0.0,32.191,0.0,2.28,622.200012
561,205.0,2020-09-19,113.599998,GFC.PA,0,0,0.0,0.0,0.0,0.0,CTO,GECINA,EUR,1.18482,1.00000,0.0,10.0,12.36,0.0,0.0,0.0,1105.39,1135.999985,30.609985,0.0,110.539,0.0,4.57,1135.999985
917,205.0,2020-09-19,26.090000,STM.PA,0,0,0.0,0.0,0.0,0.0,PEA,STMICROELECTRONICS,EUR,1.18482,1.00000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000,0.0,0.00,622.200012
1273,205.0,2020-09-19,200.389999,MSFT,0,0,0.0,0.0,0.0,0.0,CTO,MICROSOFT,USD,1.18482,1.18482,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000,0.0,0.00,1135.999985
1629,0.0,2020-09-19,0.000000,0,0,0,0.0,0.0,0.0,0.0,Cash,Cash,0,0.00000,0.00000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,8314.850000,0.000000,0.0,0.000,0.0,0.00,0.000000


# Préparation DataTable

In [160]:
df_portfolio['Close_EUR'] = df_portfolio['Close']/df_portfolio['taux_conversion']
dt_colonnes = ["Compte","Date","Symbol","Position","Cumul_Frais","PRU1","Close_EUR","Valeur Marché","latent_PL","realised_PL_cumul"]
#df_portfolio['Date']==pd.to_datetime(datetime.date(2020,9,19))
dt_ptf =df_portfolio[(df_portfolio['Date']==df_portfolio['Date'].max())&(df_portfolio['Position']>0)][dt_colonnes].sort_values(['Compte','Valeur Marché'],ascending=False)
solde1 = solde[solde['Date']==df_portfolio['Date'].max()][['Date','solde_cash']].rename(columns={'Valeur Marché':'solde_cash'})
solde1['Compte'] = 'Cash'
dt_ptf = pd.concat([dt_ptf,solde1],axis=0,sort= False)
dt_ptf[dt_ptf['Compte']=='Cash'] = dt_ptf.loc[dt_ptf['Compte']=='Cash',:].fillna(0)


In [161]:
prices = Format(precision=0,
                scheme=Scheme.fixed,
                symbol=Symbol.yes,
                symbol_suffix=u'€')
formating = [{'id':c,'name':c,'type':'text'} for c in dt_ptf.columns[:4]]+[{'id':c,'name':c,'type':'numeric','format':prices} for c in dt_ptf.columns[4:]]


# Dash Appli

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__,title="Portfolio Manager", external_stylesheets=[dbc.themes.FLATLY])


app.layout = html.Div(children=[
    dbc.Row(dbc.Col(html.H1("Portfolio Manager"),width={'size':6,'offset':3}),justify="center", align="center", className="h-50"),
    dbc.Row([dcc.DatePickerSingle(id='startDate',
                                          max_date_allowed=datetime.datetime.now(),
                                          min_date_allowed=date(2020,1,1),
                                          date=date(2020,2,1))
             ],justify="center", align="center", className="h-50"),
    
    dbc.Row([dbc.Col(dcc.Loading(id="loading-1",
                                 type="default",
                                 children=html.Div(id="loading-output")),
                     width=12)]),
    
    dbc.Row([dbc.Col(width={'size':1}),
             dbc.Col(dcc.Graph(id='stacked-area-market-value'),width={'size':5},
                     style={ 'display': 'inline-block',
                            'border-radius': '10px 0px 0px 10px',
                            'box-shadow': '4px 4px 4px grey',
                            'background-color': '#f9f9f9',
                            #'padding': '10px',
                            'margin-top':'10px',
                            'margin-bottom': '10px',
                            #'margin-left': '10px'
                           }),
             

        
        
             dbc.Col(dcc.Graph(id='stacked-area-different-values'),width={'size':5},
                    style={ 'display': 'inline-block',
                            'border-radius': '0px 10px 10px 0px',
                            'box-shadow': '4px 4px 4px grey',
                            'background-color': '#f9f9f9',
                            #'padding': '10px',
                            'margin-top':'10px',
                            'margin-bottom': '10px',
                            #'margin-left': '10px'
                           }),
            dbc.Col(width={'size':1})
            ]),#,figure=fig_soldes
    
    

    
    
    dbc.Row([dbc.Col(width={'size':1}),
             dbc.Col(
                dcc.Dropdown(id='ticker',
                             options=[{'label':'All securities','value':'Tout'},{'label':'CTO','value':'CTO'},{'label':'PEA','value':'PEA'}]+
                             [{'label':s,'value':t} for s,t in zip(list(tickers['Symbol']),list(tickers['ticker']))],
                             value='Tout'),
                width={'size':5,'offset':0}),
        
             dbc.Col(
                 dcc.Dropdown(id='variable',
                              options=[{'label':s,'value':t} for s,t in zip(['Realised PL','Cost of position','Unrealised PL','Average Cost','Fees','Position'],
                                                               ['realised_PL_cumul','Cumul','latent_PL','PRU1','Cumul_Frais','Position'])],
                              value='realised_PL_cumul'),
                 width={'size':5,'offset':0}),
            dbc.Col(width={'size':1})]),
    

    dbc.Row([dbc.Col(width={'size':1}),
             dbc.Col(dcc.Graph(id='zoom-security'),width={'size':10},
                    style={ 'display': 'inline-block',
                            'border-radius': '10px',
                            'box-shadow': '4px 4px 4px grey',
                            'background-color': '#f9f9f9',
                            #'padding': '10px',
                            'margin-top':'10px',
                            'margin-bottom': '10px',
                            #'margin-left': '10px'
                           }),
            dbc.Col(width={'size':1})]),

    dbc.Row([dbc.Col(width={'size':1}),
             dbc.Col(dash_table.DataTable(id='table',
                         columns=formating,
                         data=dt_ptf.to_dict('records'),
                        style_data_conditional=[
                        {
                            'if': {
                                'filter_query': '{{Compte}} = {}'.format('CTO'),
                            },
                            'backgroundColor': '#FF4136',
                            'color': 'white'
                        }]),width={'size':10},
                    style={ 'display': 'inline-block',
                            'border-radius': '10px',
                            'box-shadow': '4px 4px 4px grey',
                            'background-color': '#f9f9f9',
                            #'padding': '10px',
                            'margin-top':'20px',
                            'margin-bottom': '10px',
                            #'margin-left': '10px'
                           }),
            dbc.Col(width={'size':1})])
    
    
    
])


@app.callback(
    [Output("stacked-area-different-values", "figure")], 
    [Input(component_id='startDate',component_property='date')])
def portfolio_different_values(startDate):
    if startDate is not None:
        solde_global1 = solde_global[(solde_global['Date']>pd.to_datetime(startDate))]
        
        fig_soldes = px.line(solde_global1, x="Date", y="Valeur", color='Solde',
                             labels={"Valeur": "Value (€)","Date": "", "Solde":""},
                             hover_data = ['Solde'])
        fig_soldes.update_layout(legend=dict(
            orientation="h",
            yanchor="top",
            xanchor="left"))
        
        fig_soldes.update_traces(
        hovertemplate="<br>".join([
            "<b>Date</b>: %{x}",
            "<b>%{customdata[0]}</b>: %{y:.2f}€"])
        )
        fig_soldes.layout.plot_bgcolor = '#f9f9f9'
        fig_soldes.layout.paper_bgcolor = '#f9f9f9'   
        return [fig_soldes]
    
    
    
@app.callback(
    [Output("loading-output", "children"),
     Output("stacked-area-market-value", "figure")], 
    [Input(component_id='startDate',component_property='date')])
def portfolio_market_value(startDate):
    if startDate is not None:
        df_portfolio3 = df_portfolio2[(df_portfolio2['Date']>pd.to_datetime(startDate))]
        
        fig = px.area(df_portfolio3, 
                      x=df_portfolio3['Date'], 
                      y=df_portfolio3['Valeur Marché'],
                      color=df_portfolio3['Compte'],
                      line_group=df_portfolio3['ticker'],
                      labels={"Valeur Marché": "Market Value (€)","Date": ""},
                      #title="Manually Specified Labels",
                      hover_data=["PRU1","Position","Close","realised_PL_cumul","Symbol","latent_PL","Evaluation"])
        
            
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(0,0,0,0)'
        ))
        fig.update_traces(
            hovertemplate="<br>".join([
            "<b>Security</b>: %{customdata[4]}",
            "<b>Date</b>: %{x}",
            "<b>Valeur Marché</b>: %{y:.2f}€",
            "<b>PRU</b>: %{customdata[0]:.2f}€",
            "<b>Position</b>: %{customdata[1]}",
            "<b>Realised PL</b>: %{customdata[3]:.2f}€",
            "<b>Unrealised PL</b>: %{customdata[5]:.2f}€",
            "<b>Evaluation PTF</b>: %{customdata[6]:.2f}€"])
        )
        fig.layout.plot_bgcolor = '#f9f9f9'
        fig.layout.paper_bgcolor = '#f9f9f9'
        
        return ["",fig]
    
    
    
    
@app.callback(
    Output("zoom-security", "figure"), 
    [Input(component_id='ticker',component_property='value'),
    Input(component_id='variable',component_property='value')])
def zoom_on_positions(ticker,variable):
    if ticker=='Tout':
        df_portfolio3 = df_portfolio2
    elif ticker=='CTO' or ticker=='PEA':
        df_portfolio3 = df_portfolio2[(df_portfolio2['Compte']==ticker)]
    else:
        df_portfolio3 = df_portfolio2[(df_portfolio2['Compte']!='Cash')&(df_portfolio2['ticker']==ticker)]
    fig = px.area(df_portfolio3, 
                  x=df_portfolio3['Date'], 
                  y=df_portfolio3[variable],
                  color=df_portfolio3['ticker'],
                  labels={variable: variable.replace("_"," ")+" (€)","Date": ""},
                  hover_data=["PRU1", "Position","Symbol"])
    
    fig.update_layout(legend=dict(
            orientation="h",
            yanchor="top",
            xanchor="left"))
    
    fig.update_traces(
        hovertemplate="<br>".join([
            "<b>Security</b>: %{customdata[2]}",
            "<b>Date</b>: %{x}",
            "<b>{variable}</b>:".format(variable=variable)+" %{y:.2f}€",
            "<b>PRU</b>: %{customdata[0]:.2f}€",
            "<b>Position</b>: %{customdata[1]}"])
        )
    
    fig.layout.plot_bgcolor = '#f9f9f9'
    fig.layout.paper_bgcolor = '#f9f9f9'
        
    return fig



if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Feb/2021 00:32:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_0m1613340017.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_0m1613340016.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_0m1613340017.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_0m1613340016.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-suites/dash_table/bundle.v4_11_2m1613340017.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_15_0m1613340017.min.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2021 00:32:18] "GET /_dash-component-